## Load notebooks from github

This cell will download notebooks and put them in the **examples** directory in two subdirectories:
* service - notebooks for use with the cuOpt service
* sdk - notebooks for use with the cuOpt Python SDK


In [ ]:
%%bash
mkdir -p examples/service
mkdir examples/sdk
git clone  -b branch-25.02 --single-branch https://github.com/NVIDIA/cuopt-examples
mv cuopt-examples/LICENSE.md cuopt-examples/README.md ./examples

# Service notebooks
for d in cuFOLIO_portfolio_optimization sample_lp_sever_notebooks routing_optimization_over_server; do
    mv cuopt-examples/$d ./examples/service/
    if [ -f ./examples/service/$d/requirements.txt ]; then
        pip install -q -r ./examples/service/$d/requirements.txt
    fi
done

# SDK notebooks
for d in last_mile_delivery intra-factory_transport; do
    mv cuopt-examples/$d ./examples/sdk/
    if [ -f ./examples/sdk/$d/requirements.txt ]; then
        pip install -q -r ./examples/sdk/$d/requirements.txt
    fi
done
rm -rf cuopt-examples

One-time setup for the service notebooks: [Install and run the cuOpt server](#cuopt-server)

One-time setup for the SDK notebooks: [Install the cuOpt Python SDK](#cuopt-sdk)

After you have performed the setup, return to the file navigation pane to run the example notebooks

<a id="cuopt-server"></a>
# Install and run the cuOpt server

This is only necessary if you are going to run the notebooks under **examples/service**

## Install the cuOpt self-hosted client

In [ ]:
%%bash
pip install cuopt_sh_client --extra-index-url=https://pypi.nvidia.com

## Add the current user to the docker group

#### This requires passwordless sudo and allows the current user to run containers

In [ ]:
%%bash
sudo usermod -aG docker $USER
sudo su $USER

## Enter a NGC api key and download the cuOpt container image
### Note: you must advance manually to the next cell after successfully setting an api key

In [ ]:
import getpass, os
os.environ["CUOPT_IMAGE"] = "nvcr.io/nvidia/cuopt/cuopt-ea:25.02"
if os.environ.get("NGC_API_KEY"):
    p = getpass.getpass("You have an api key set already. Enter a new api key or simply press enter to keep the current key: ")
    if p:
        os.environ["NGC_API_KEY"] = p
        print("Successfully set api key.")        
    else:
        print("Current api key kept")
else:
    os.environ["NGC_API_KEY"] = getpass.getpass("Enter your api key and press enter: ")
    if os.environ["NGC_API_KEY"]:
        print("Successfully set api key.")

In [ ]:
%%bash
docker logout nvcr.io
echo "${NGC_API_KEY}" | docker login nvcr.io -u '$oauthtoken' --password-stdin
echo pulling image ....
docker pull $CUOPT_IMAGE
docker logout nvcr.io

In [ ]:
del os.environ['NGC_API_KEY']

## Starting and Stopping the cuOpt server

Now that the container is installed you may start and stop the cuOpt server at any time with the following cells

### Start server

In [ ]:
import subprocess
import os

# Make sure we do not already have the cuopt server running
docker_cmd = [
    "docker", "ps"
]
ps = subprocess.run(docker_cmd, capture_output=True, text=True)
for l in ps.stdout.split("\n"):
    if "cuopt" in l:
        container_id = l.split()[0]
        print("cuopt is already running.")    
        print(container_id)
        break
else:
    docker_cmd = [
        "docker", "run",
        "--rm", "-d",
        "--network=host",
        "--gpus=1", "-it",
        os.environ['CUOPT_IMAGE']
    ]
    container_id = subprocess.run(docker_cmd, capture_output=True, text=True).stdout.strip()
    print("cuopt started")
    print(container_id)



### Stop server

In [ ]:
import subprocess
import os

docker_cmd = [
    "docker", "stop", container_id
]
ps = subprocess.run(docker_cmd, capture_output=True, text=True)
if ps.stderr:
    print(ps.stderr)
else:
    print("cuopt server stopped.")

<a id="cuopt-sdk"></a>
# Install the cuOpt Python SDK

This is only necessary if you are going to run the notebooks under **examples/sdk**

The following cell installs some libraries that may not be present along with the cuOpt Python SDK

In [ ]:
CUDA_VERSION = "12"
CUDA_SUFFIX = f"cu{CUDA_VERSION}"

In [ ]:
%%bash -s "$CUDA_VERSION" "$CUDA_SUFFIX"

# Configuration
CUDA_VERSION="$1"
CUDA_SUFFIX="$2"

# Function to check if a library exists
check_lib() {
    ldconfig -p | grep -q "$1"
    return $?
}

# Function to find library path
find_lib() {
    sudo find / -name "$1" 2>/dev/null | head -n 1
}

# Function to add path to ld.so.conf
add_to_ldconfig() {
    local lib_dir="$1"
    if [ -n "$lib_dir" ]; then
        echo "$lib_dir" | sudo tee -a /etc/ld.so.conf.d/cuda-${CUDA_SUFFIX}.conf >/dev/null
    fi
}

echo "Setting up CUDA ${CUDA_VERSION} libraries..."

# Install and setup CUDART if needed
if ! check_lib "libcudart.so.${CUDA_VERSION}"; then
    echo "Installing nvidia-cuda-runtime-${CUDA_SUFFIX}..."
    sudo pip3 install nvidia-cuda-runtime-${CUDA_SUFFIX}
    
    # Find CUDART library
    CUDART_PATH=$(find_lib "libcudart.so.${CUDA_VERSION}")
    if [ -z "$CUDART_PATH" ]; then
        echo "Error: Could not find libcudart.so.${CUDA_VERSION}"
        exit 1
    fi
    
    # Create symlink for libcudart.so
    CUDART_DIR=$(dirname "$CUDART_PATH")
    if [ ! -f "${CUDART_DIR}/libcudart.so" ]; then
        echo "Creating symlink for libcudart.so..."
        sudo ln -s "$CUDART_PATH" "${CUDART_DIR}/libcudart.so"
    fi
    
    # Add to LD_LIBRARY_PATH
    add_to_ldconfig "$CUDART_DIR"
else
    echo "CUDART library already installed"
fi

# Install and setup NVRTC if needed
if ! check_lib "libnvrtc.so.${CUDA_VERSION}"; then
    echo "Installing nvidia-cuda-nvrtc-${CUDA_SUFFIX}..."
    sudo pip3 install nvidia-cuda-nvrtc-${CUDA_SUFFIX}
    
    # Find NVRTC library
    NVRTC_PATH=$(find_lib "libnvrtc.so.${CUDA_VERSION}")
    if [ -z "$NVRTC_PATH" ]; then
        echo "Error: Could not find libnvrtc.so.${CUDA_VERSION}"
        exit 1
    fi
    
    # Add to LD_LIBRARY_PATH
    NVRTC_DIR=$(dirname "$NVRTC_PATH")
    add_to_ldconfig "$NVRTC_DIR"
else
    echo "NVRTC library already installed"
fi

In [ ]:
%%bash
echo "Updating ldconfig..."
sudo ldconfig

In [ ]:
%%bash -s "$CUDA_SUFFIX"
echo "Install cudf..."
pip3 install cudf-$1

In [ ]:
%%bash
echo "Install cuOpt..."
pip3 install /home/ubuntu/*.whl

### COPYRIGHT and LICENSE

SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
SPDX-License-Identifier: Apache-2.0

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
